#Capítulo 6 - Visualizações Clássicas Multivariadas

Instalação do plotly e do renderer Orca, para que o Github apresente as imagens geradas pelo Plotly

In [ ]:
!pip install plotly==4.14.3
!pip install orca
!apt-get install xvfb libgtk2.0-0 libgconf-2-4

     |████████████████████████████████| 13.2MB 277kB/s 
  Found existing installation: plotly 4.4.1
    Uninstalling plotly-4.4.1:
      Successfully uninstalled plotly-4.4.1
     |████████████████████████████████| 245kB 5.5MB/s 
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  gconf-service gconf-service-backend gconf2-common libdbus-glib-1-2
  libgail-common libgail18 libgtk2.0-bin libgtk2.0-common
Suggested packages:
  gvfs
The following NEW packages will be installed:
  gconf-service gconf-service-backend gconf2-common libdbus-glib-1-2
  libgail-common libgail18 libgconf-2-4 libgtk2.0-0 libgtk2.0-bin
  libgtk2.0-common xvfb
0 upgraded, 11 newly installed, 0 to remove and 31 not upgraded.
Need to get 3,715 kB of archives.
After this operation, 17.2 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 libdbus-glib-1-2 amd64 0.110-2 [58.3 

Bibliotecas importantes

In [ ]:
import pandas as pd
from sklearn import preprocessing
#from ipywidgets import interactive, HBox, VBox
import plotly.express as px
#from plotly.offline import init_notebook_mode, iplot
##import plotly.offline as py
import plotly.graph_objects as go
import numpy as np
from sklearn.decomposition import PCA as sklearnPCA
from sklearn import manifold
from sklearn.neighbors import NeighborhoodComponentsAnalysis

Abre o arquivo .data utilizando o Pandas. O conjunto auto-mpg pode ser baixado aqui:


https://archive.ics.uci.edu/ml/datasets/Auto+MPG

In [ ]:
import seaborn as sns

raw_data = sns.load_dataset('tips')

In [ ]:
raw_data.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


Informações sobre os atributos


In [ ]:
raw_data = raw_data.replace('?',np.NaN)
raw_data = raw_data.replace('NA',np.NaN)
raw_data = raw_data.dropna(axis=0)

In [ ]:
label = raw_data["sex"]
#label_int = raw_data["sex"]


#raw_data = raw_data.drop(labels=["sex"],axis=1)

raw_data

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4
...,...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,Dinner,3
240,27.18,2.00,Female,Yes,Sat,Dinner,2
241,22.67,2.00,Male,Yes,Sat,Dinner,2
242,17.82,1.75,Male,No,Sat,Dinner,2


In [ ]:
#import plotly.io as pio
#pio.renderers.default = 'png'
import plotly.express as px
df = raw_data.copy()
df["sex"] = label

fig = px.scatter(df, x="total_bill", y="tip",color="sex")
fig.show()

Matriz de Scatterplot

In [ ]:
import plotly.figure_factory as ff
# prepare data

selected_data = raw_data.loc[:,["tip","size","smoker","day","time"]]
selected_data["index"] = np.arange(1,len(selected_data)+1)
# scatter matrix
fig = ff.create_scatterplotmatrix(selected_data, diag='box', index='index',colormap='Portland',
                                  colormap_type='cat',
                                  height=600, width=600)
fig.show()
#iplot(fig)

Agora utilizando o go.Scatter

In [ ]:
trace = go.Scatter(x = df['total_bill'],
                   y = df['tip'],
                   marker_color = df["size"],
                   mode = 'markers')

vis = [trace]
layout = go.Layout(title='Cor = Size',
                   yaxis={'title':'tip'},
                   xaxis={'title': 'size'})
fig = go.Figure(data=vis, layout=layout)
fig.show()
#py.iplot(fig)

## Visualização baseada no posicionamento de pontos utilizando Análise de Componentes Principais

Reduz a dimensionalidade dos dados

In [ ]:
raw_data

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4
...,...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,Dinner,3
240,27.18,2.00,Female,Yes,Sat,Dinner,2
241,22.67,2.00,Male,Yes,Sat,Dinner,2
242,17.82,1.75,Male,No,Sat,Dinner,2


In [ ]:
label = raw_data["sex"]
raw_data = raw_data.drop(labels=["sex"],axis=1)

In [ ]:
dataset = raw_data
dataset = dataset.replace('No',0)
dataset = dataset.replace('Yes',1)
#dataset = dataset.replace('Male',0)
#dataset = dataset.replace('Female',1)
dataset = dataset.replace('Lunch',0)
dataset = dataset.replace('Dinner',1)
dataset = dataset.replace('Sun',0)
dataset = dataset.replace('Sat',1)
dataset = dataset.replace('Thur',0.33)
dataset = dataset.replace('Fri',0.66)
X = dataset.values
y = label.values
a = dataset.columns
#normalize = preprocessing.MinMaxScaler()
#xscaled = normalize.fit_transform(X)
scaler = preprocessing.StandardScaler().fit(X)
xscaled = scaler.transform(X)
dataset = pd.DataFrame(xscaled,columns=a)
dataset

,total_bill,tip,smoker,day,time,size
0,-0.314711,-1.439947,-0.784789,-1.171366,0.621582,-0.600193
1,-1.063235,-0.969205,-0.784789,-1.171366,0.621582,0.453383
2,0.137780,0.363356,-0.784789,-1.171366,0.621582,0.453383
3,0.438315,0.225754,-0.784789,-1.171366,0.621582,-0.600193
4,0.540745,0.443020,-0.784789,-1.171366,0.621582,1.506958
...,...,...,...,...,...,...
239,1.040511,2.115963,-0.784789,1.210411,0.621582,0.453383
240,0.832275,-0.722971,1.274228,1.210411,0.621582,-0.600193
241,0.324630,-0.722971,1.274228,1.210411,0.621582,-0.600193
242,-0.221287,-0.904026,-0.784789,1.210411,0.621582,-0.600193


In [ ]:
X = dataset.values
y = label.values

sklearn_pca = sklearnPCA(n_components=2)
X_embedded = sklearn_pca.fit_transform(X)

#pca = PCA(n_components=2)
#components = pca.fit_transform(X)

fig = px.scatter(X_embedded, x=0, y=1, color=label.astype(object))
fig.show()

In [ ]:
xisomap = manifold.Isomap(10, 2).fit_transform(X)
fig = px.scatter(xisomap, x=0, y=1, color=label.astype(object))
fig.show()

In [ ]:
mds = manifold.MDS(2, max_iter=100, n_init=1)
xmds = mds.fit_transform(X)
fig = px.scatter(xmds, x=0, y=1, color=label.astype(object))
fig.show()

In [ ]:
se = manifold.SpectralEmbedding(n_components=2,
                                n_neighbors=10)
xspec = se.fit_transform(X)
fig = px.scatter(xspec, x=0, y=1, color=label.astype(object))
fig.show()

In [ ]:
tsne = manifold.TSNE(n_components=2, init='pca', random_state=0)
xtsne = tsne.fit_transform(X)
fig = px.scatter(xtsne, x=0, y=1, color=label.astype(object))
fig.show()

In [ ]:
nca = NeighborhoodComponentsAnalysis(max_iter=15, random_state=0)
nca = nca.fit(xtsne,y)
xneig = nca.transform(xtsne)
fig = px.scatter(xneig, x=0, y=1, color=label.astype(object))
fig.show()

In [ ]:
saida = manifold.trustworthiness(X, xtsne, n_neighbors=50, metric='euclidean')
saida

0.9094542005156394

## Gráficos de linha

Vamos gerar um gráfico de linhas para o conjunto de dados Auto-MPG Cars. Considere que cada carro é enumerado por um número inteiro de 1 a N, em que N é a quantidade de instâncias. Agora vamos criar uma nova coluna no DataFrame para possibilitar seu uso como eixo X no gráfico de linhas (Line Chart) do Plotly:

Em seguida, podemos criar um gráfico de linhas:

In [ ]:
raw_data = raw_data.sort_values(by=['total_bill']);
fig = px.line(raw_data, x="size", y="total_bill", title='Total_Bill vs Size')
fig.show()

Vamos gerar outro gráfico de linhas, agora considerando a ordenação do atributo de preço médio do veículo (displacement):


In [ ]:
raw_data = raw_data.sort_values(by=['tip']);
fig = px.line(raw_data, x="size", y="tip", title='Tip vs Size')
fig.show()

## Coordenadas Paralelas

In [ ]:
fig = px.parallel_coordinates(raw_data, color="size",
                              dimensions=list(raw_data.columns))
fig.show()

In [ ]:
fig = px.parallel_categories(raw_data, color="size", color_continuous_scale=px.colors.sequential.Inferno)
fig.show()

In [ ]:
fig = px.bar(raw_data, x="sex", y="total_bill", color="smoker", barmode="group")
fig.show()

In [ ]:
fig = px.histogram(raw_data, x="total_bill", y="tip", color="sex", marginal="rug", hover_data=raw_data.columns)
fig.show()